In [1]:
%matplotlib inline
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# For data preprocessing
from tensorflow.keras.utils import to_categorical

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [2]:
df = pd.read_csv('euw.csv')

In [3]:
df.drop(df.columns[0], axis=1, inplace=True)

In [4]:
x_train = df[df.columns[1:]]
y_train = df["result"]

In [20]:
x_train

,top_win_rate,top_lane_kill_rate,top_kda,top_kill_participate,top_damage,jg_win_rate,jg_kda,jg_kill_participate,jg_damage,mid_win_rate,...,mid_damage,adc_win_rate,adc_lane_kill_rate,adc_kda,adc_kill_participate,adc_damage,sup_win_rate,sup_lane_kill_rate,sup_kda,sup_kill_participate
0,0.4849,0.4237,0.513369,0.513716,0.487368,0.4936,0.499002,0.502503,0.469126,0.4875,...,0.468713,0.5423,0.4937,0.528155,0.516958,0.516174,0.4953,0.5846,0.449275,0.478774
1,0.5014,0.5535,0.485714,0.482718,0.515910,0.4980,0.499002,0.507630,0.486446,0.5556,...,0.572448,0.5036,0.5891,0.527451,0.525762,0.512016,0.4953,0.4914,0.478516,0.475356
2,0.5082,0.6148,0.593381,0.524331,0.516326,0.5107,0.522449,0.518191,0.447081,0.5147,...,0.424833,0.4779,0.5416,0.482243,0.494781,0.456389,0.4724,0.4548,0.409814,0.480403
3,0.5122,0.3297,0.400763,0.482803,0.591403,0.5412,0.515658,0.507614,0.481548,0.5665,...,0.502442,0.5017,0.4662,0.479125,0.490216,0.463594,0.5432,0.3825,0.501873,0.530335
4,0.4865,0.5265,0.553299,0.533696,0.503255,0.4967,0.466377,0.510121,0.493189,0.5578,...,0.541907,0.4964,0.4109,0.472549,0.474238,0.487984,0.5105,0.4542,0.494983,0.509524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.5037,0.4119,0.437975,0.481572,0.478134,0.4867,0.428571,0.488417,0.467855,0.4899,...,0.420969,0.4837,0.4926,0.436893,0.486430,0.469174,0.4906,0.4172,0.481216,0.393983
2861,0.5089,0.5269,0.570194,0.521429,0.497815,0.4907,0.446397,0.490079,0.606387,0.5388,...,0.533293,0.4854,0.3485,0.397490,0.464968,0.460317,0.4951,0.6011,0.493601,0.462641
2862,0.6306,0.4598,0.603734,0.539928,0.505202,0.5020,0.500998,0.492370,0.513554,0.4835,...,0.477617,0.4998,0.4270,0.534783,0.506098,0.491764,0.5546,0.3859,0.496689,0.531477
2863,0.5593,0.6230,0.568928,0.501788,0.437633,0.5270,0.489945,0.480569,0.607274,0.5408,...,0.527127,0.4926,0.4956,0.445808,0.479879,0.440301,0.5362,0.4975,0.588608,0.502518


In [6]:
y_train.shape

(2865,)

In [7]:
y_train = to_categorical(y_train, 2)

In [8]:
y_train[1003]

array([1., 0.], dtype=float32)

In [9]:
FC_layers = [Dense(units=256, input_dim=23, activation='relu'),
             Dense(units=128, activation = 'relu'),
             Dense(units=2, activation='softmax')]

In [10]:
model = Sequential(FC_layers)
# model.add(Dense(87, input_dim=784, activation='relu'))
# model.add(Dense(128,activation = 'relu'))
# model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 39,298
Trainable params: 39,298
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, batch_size=50, epochs=50, validation_split = 1/6)

Train on 2387 samples, validate on 478 samples
Epoch 1/50
2387/2387 [==============================] - 0s 65us/sample - loss: 0.2495 - accuracy: 0.5245 - val_loss: 0.2513 - val_accuracy: 0.5021
Epoch 2/50
2387/2387 [==============================] - 0s 74us/sample - loss: 0.2497 - accuracy: 0.5178 - val_loss: 0.2506 - val_accuracy: 0.5000
Epoch 3/50
2387/2387 [==============================] - 0s 67us/sample - loss: 0.2491 - accuracy: 0.5266 - val_loss: 0.2507 - val_accuracy: 0.4623
Epoch 4/50
2387/2387 [==============================] - 0s 67us/sample - loss: 0.2494 - accuracy: 0.5224 - val_loss: 0.2516 - val_accuracy: 0.5021
Epoch 5/50
2387/2387 [==============================] - 0s 61us/sample - loss: 0.2496 - accuracy: 0.5249 - val_loss: 0.2507 - val_accuracy: 0.4937
Epoch 6/50
2387/2387 [==============================] - 0s 65us/sample - loss: 0.2491 - accuracy: 0.5207 - val_loss: 0.2516 - val_accuracy: 0.5021
Epoch 7/50
2387/2387 [==============================] - 0s 60us/sample 

In [21]:
model.save('model1.h5')